In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
import sklearn
import data
import pool
import time
import pooltool as pt

In [4]:
df

,cue_x,cue_y,one_x,one_y,two_x,two_y,speed,vspin,hspin
0,0.181676,0.575535,0.731904,1.005585,0.606757,0.484032,2.60,0.0,0.0
1,0.202868,0.084844,0.312938,1.063488,0.574457,0.178813,2.30,0.4,0.1
2,0.194066,0.480850,0.580834,1.361406,0.798477,1.588291,1.55,0.1,0.0
3,0.082032,0.376628,0.700692,1.690720,0.916421,0.573849,1.40,0.0,0.0
4,0.182712,0.850552,0.929994,0.367181,0.487540,0.400771,0.80,0.0,-0.2
...,...,...,...,...,...,...,...,...,...
195,0.399202,0.644537,0.153489,1.119295,0.403151,1.062046,1.40,0.3,0.3
196,0.355118,0.851031,0.108906,0.882790,0.400198,0.564125,1.25,0.0,0.2
197,0.330939,0.538350,0.650280,0.069279,0.492718,1.467742,1.40,0.3,0.0
198,0.158199,0.432935,0.697680,1.070156,0.836957,0.332437,0.80,0.0,0.0


In [19]:
steps = 200

start = time.time()
df = data.get_hit_data(steps)
print(f'{(time.time()-start)/steps} seconds per step')

10 steps done
20 steps done
30 steps done
40 steps done
50 steps done
60 steps done
70 steps done
80 steps done
90 steps done
100 steps done
110 steps done
120 steps done
130 steps done
140 steps done
150 steps done
160 steps done
170 steps done
180 steps done
190 steps done
200 steps done
95.5825700199604 seconds per step


In [20]:
cdf

,cue_x,cue_y,one_x,one_y,two_x,two_y,speed,vspin,hspin
0,0.233700,0.647571,0.402179,0.284573,0.655099,0.329529,1.25,-0.5,0.0
1,0.079220,0.417230,0.346198,0.172842,0.887643,0.215223,0.95,0.0,0.0
2,0.191561,0.959704,0.338511,0.541561,0.085335,1.738272,2.00,0.0,0.0
3,0.239587,0.573218,0.571472,1.133612,0.032145,1.892367,1.55,-0.3,0.0
4,0.229396,0.524396,0.197874,1.516775,0.591974,0.549463,1.70,0.0,-0.3
...,...,...,...,...,...,...,...,...,...
15595,0.105762,0.918199,0.318486,0.248407,0.798430,0.674730,1.70,0.1,-0.1
15596,0.366726,0.556364,0.293317,0.898841,0.098640,0.549313,1.55,0.1,0.0
15597,0.080874,0.404310,0.843240,0.771046,0.736955,0.053003,1.55,-0.1,-0.1
15598,0.180083,0.509725,0.260309,0.957844,0.646462,0.522819,2.15,-0.5,0.0


In [22]:
cdf = pd.concat([df, cdf])
cdf

,cue_x,cue_y,one_x,one_y,two_x,two_y,speed,vspin,hspin
0,0.231092,0.805833,0.810622,0.563653,0.255239,1.798254,1.70,-0.1,0.1
1,0.150648,0.351235,0.819565,1.267361,0.431713,1.077468,0.80,-0.4,-0.2
2,0.170627,0.652094,0.179184,1.690016,0.960380,0.957741,2.00,-0.2,0.0
3,0.218175,0.476620,0.813862,1.191725,0.403459,1.871007,2.00,0.1,0.2
4,0.426315,0.862200,0.562864,0.165306,0.306319,0.549656,1.85,-0.1,-0.2
...,...,...,...,...,...,...,...,...,...
16295,0.105762,0.918199,0.318486,0.248407,0.798430,0.674730,1.70,0.1,-0.1
16296,0.366726,0.556364,0.293317,0.898841,0.098640,0.549313,1.55,0.1,0.0
16297,0.080874,0.404310,0.843240,0.771046,0.736955,0.053003,1.55,-0.1,-0.1
16298,0.180083,0.509725,0.260309,0.957844,0.646462,0.522819,2.15,-0.5,0.0


In [23]:
cdf.to_csv('Datasets/two_ball_hit_new.csv', index = False)

In [78]:
cdf = pd.read_csv('Datasets/two_ball_hit_new.csv')

In [3]:
reg_net = torch.load('reg_net.pt')
# reg_net_2 = torch.load('reg_net_2.pt')
# reg_net_2b = torch.load('reg_net_2b.pt')
# reg_net_2c = torch.load('reg_net_2c.pt')

In [24]:
df0 = pd.read_csv('Datasets/one_ball_full_pos0.csv')
df1 = pd.read_csv('Datasets/one_ball_full_pos1.csv')
df2 = pd.read_csv('Datasets/one_ball_full_pos2.csv')
df3 = pd.read_csv('Datasets/one_ball_full_pos3.csv')

In [33]:
df0

,cue_x,cue_y,one_x,one_y,speed,vspin,hspin,phi,lb,lc,lt,rb,rc,rt,end_x,end_y
0,0.194281,0.276859,0.046968,0.868859,3.352847,-0.297846,-0.228028,103.059511,0.0,1.0,0.0,0.0,0.0,0.0,0.143499,0.285342
1,0.110727,0.211206,0.303572,0.722603,0.953745,-0.081331,-0.107463,70.205526,0.0,0.0,0.0,0.0,0.0,1.0,0.277049,1.373292
2,0.039058,0.409307,0.494349,0.761136,3.361030,0.285583,0.146790,39.119940,0.0,0.0,0.0,0.0,1.0,0.0,0.093820,0.367331
3,0.154962,0.297496,0.331983,0.098922,1.299785,0.224232,-0.201314,-58.014290,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000
4,0.162361,0.046366,0.325928,0.921265,2.326245,-0.355351,-0.142924,80.814532,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988436,0.417280,0.639539,0.444617,0.362546,2.218609,0.456062,0.277925,-94.813828,0.0,0.0,0.0,1.0,0.0,0.0,0.301614,0.201524
988437,0.246293,0.930422,0.372301,0.685645,2.012820,0.093833,-0.180259,-66.521064,0.0,0.0,0.0,1.0,0.0,0.0,0.412886,0.141335
988438,0.230222,0.917824,0.308533,0.147827,2.080515,0.385073,0.285081,-80.064365,1.0,0.0,0.0,0.0,0.0,0.0,0.550077,0.313754
988439,0.077444,0.747667,0.134962,0.507670,2.009560,-0.169764,0.019768,-69.281787,1.0,0.0,0.0,0.0,0.0,0.0,0.531362,0.208529


In [79]:
cdf

,cue_x,cue_y,one_x,one_y,two_x,two_y,speed,vspin,hspin
0,0.231092,0.805833,0.810622,0.563653,0.255239,1.798254,1.70,-0.1,0.1
1,0.150648,0.351235,0.819565,1.267361,0.431713,1.077468,0.80,-0.4,-0.2
2,0.170627,0.652094,0.179184,1.690016,0.960380,0.957741,2.00,-0.2,0.0
3,0.218175,0.476620,0.813862,1.191725,0.403459,1.871007,2.00,0.1,0.2
4,0.426315,0.862200,0.562864,0.165306,0.306319,0.549656,1.85,-0.1,-0.2
...,...,...,...,...,...,...,...,...,...
16495,0.105762,0.918199,0.318486,0.248407,0.798430,0.674730,1.70,0.1,-0.1
16496,0.366726,0.556364,0.293317,0.898841,0.098640,0.549313,1.55,0.1,0.0
16497,0.080874,0.404310,0.843240,0.771046,0.736955,0.053003,1.55,-0.1,-0.1
16498,0.180083,0.509725,0.260309,0.957844,0.646462,0.522819,2.15,-0.5,0.0


In [9]:
cdf = pd.read_csv('Datasets/two_ball_hit.csv')

In [38]:
df0.to_csv('C:/Users/Masahiro/Datasets/one_ball_full_pos0.csv', index = False)
df1.to_csv('C:/Users/Masahiro/Datasets/one_ball_full_pos1.csv', index = False)
df2.to_csv('C:/Users/Masahiro/Datasets/one_ball_full_pos2.csv', index = False)
df3.to_csv('C:/Users/Masahiro/Datasets/one_ball_full_pos3.csv', index = False)

In [37]:
print(len(df1.query('cue_y > 0.5')), len(df1.query('cue_y < 0.5')))

1069512 1086764


In [27]:
cdf0 = cdf.query('one_x < 0.5 and one_y < 1')
cdf1 = cdf.query('one_x < 0.5 and one_y > 1')
cdf2 = cdf.query('one_x > 0.5 and one_y < 1')
cdf3 = cdf.query('one_x > 0.5 and one_y > 1')

In [28]:
df0 = pd.concat([df0, cdf0])
df1 = pd.concat([df1, cdf1])
df2 = pd.concat([df2, cdf2])
df3 = pd.concat([df3, cdf3])

In [39]:
cdf = pd.concat([df0, df1, df2, df3])

In [77]:
cdf

,cue_x,cue_y,one_x,one_y,two_x,two_y,speed,vspin,hspin
0,0.231092,0.805833,0.810622,0.563653,0.255239,1.798254,1.70,-0.1,0.1
1,0.150648,0.351235,0.819565,1.267361,0.431713,1.077468,0.80,-0.4,-0.2
2,0.170627,0.652094,0.179184,1.690016,0.960380,0.957741,2.00,-0.2,0.0
3,0.218175,0.476620,0.813862,1.191725,0.403459,1.871007,2.00,0.1,0.2
4,0.426315,0.862200,0.562864,0.165306,0.306319,0.549656,1.85,-0.1,-0.2
...,...,...,...,...,...,...,...,...,...
16295,0.105762,0.918199,0.318486,0.248407,0.798430,0.674730,1.70,0.1,-0.1
16296,0.366726,0.556364,0.293317,0.898841,0.098640,0.549313,1.55,0.1,0.0
16297,0.080874,0.404310,0.843240,0.771046,0.736955,0.053003,1.55,-0.1,-0.1
16298,0.180083,0.509725,0.260309,0.957844,0.646462,0.522819,2.15,-0.5,0.0


In [80]:
X = cdf.loc[:,[col not in ['speed', 'vspin', 'hspin'] for col in cdf.columns]]
y = cdf[['speed', 'vspin', 'hspin']]
#y = cdf[['speed']]
#y = cdf[['hspin']]

In [38]:
X = df3.loc[:,[col not in ['end_x', 'end_y'] for col in df.columns]]
y = df3[['end_x', 'end_y']]

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

In [86]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
dall = xgb.DMatrix(X, label=y)

In [87]:
param = {'max_depth': 30, 'eta': 0.5, 'lambda': 100, 'objective': 'reg:squarederror'}
evallist = [(dall, 'train'), (dtest, 'eval')]
num_round = 30

reg = xgb.train(param, dall, num_round, evallist)

[0]	train-rmse:0.53597	eval-rmse:0.53558
[1]	train-rmse:0.39369	eval-rmse:0.39177
[2]	train-rmse:0.34413	eval-rmse:0.34150
[3]	train-rmse:0.31694	eval-rmse:0.31440
[4]	train-rmse:0.29946	eval-rmse:0.29681
[5]	train-rmse:0.28629	eval-rmse:0.28406
[6]	train-rmse:0.27384	eval-rmse:0.27220
[7]	train-rmse:0.26287	eval-rmse:0.26212
[8]	train-rmse:0.25354	eval-rmse:0.25230
[9]	train-rmse:0.24497	eval-rmse:0.24352
[10]	train-rmse:0.23634	eval-rmse:0.23505
[11]	train-rmse:0.22861	eval-rmse:0.22751
[12]	train-rmse:0.22125	eval-rmse:0.21973
[13]	train-rmse:0.21414	eval-rmse:0.21236
[14]	train-rmse:0.20780	eval-rmse:0.20624
[15]	train-rmse:0.20120	eval-rmse:0.19951
[16]	train-rmse:0.19508	eval-rmse:0.19366
[17]	train-rmse:0.18961	eval-rmse:0.18827
[18]	train-rmse:0.18396	eval-rmse:0.18269
[19]	train-rmse:0.17880	eval-rmse:0.17772
[20]	train-rmse:0.17330	eval-rmse:0.17223
[21]	train-rmse:0.16819	eval-rmse:0.16730
[22]	train-rmse:0.16341	eval-rmse:0.16245
[23]	train-rmse:0.15905	eval-rmse:0.15813
[2

In [199]:
reg = xgb.XGBRegressor(param)

reg.load_model('boosted.json')

C:\Users\Masahiro\anaconda3\envs\ML\lib\site-packages\xgboost\core.py:617: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)
C:\Users\Masahiro\anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py:782: UserWarning: Loading a native XGBoost model with Scikit-Learn interface.
  warnings.warn("Loading a native XGBoost model with Scikit-Learn interface.")


In [22]:
reg = xgb.XGBRegressor(n_estimators = 4000)
reg.fit(X_train, y_train)

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:pocket: object

In [42]:
#Construct an ensemble of boosted trees

n_trees = 4
ensemble = []

X_train = X_train.reset_index(drop = True)
y_train = y_train.reset_index(drop = True)

for i in range(n_trees):
    X_sample = X_train.sample(10000, replace=True)
    print('finished sampling')
    y_sample = y_train.iloc[X_sample.index]
    reg = xgb.XGBRegressor(n_estimators = 2000)
    reg.fit(X_sample, y_sample)
    ensemble.append(reg)
    print('finished one tree')


finished sampling
finished one tree
finished sampling
finished one tree
finished sampling
finished one tree
finished sampling
finished one tree


In [88]:
reg.save_model('boosted.json')

In [47]:
#Test ensemble
    
pred = 0

for tree in ensemble:
    print(tree.score(X_test, y_test))
    pred += tree.predict(X_test)
    

pred = pred/(len(ensemble))


sklearn.metrics.mean_squared_error(y_test, pred)

-0.27348513598823887
-0.27246003380502765
-0.2810494551178302
-0.27167957588678987


0.21497103015279626

In [48]:
pred

array([[ 1.2633480e+00,  2.9607646e-02, -1.6741149e-02],
       [ 2.3976054e+00,  1.8321882e-01, -1.5047547e-01],
       [ 1.6544925e+00, -7.2578155e-03,  7.4980594e-02],
       ...,
       [ 2.3123200e+00,  8.3500244e-02, -2.6723787e-02],
       [ 1.6825007e+00,  2.3178117e-02, -7.0519723e-02],
       [ 1.5403135e+00,  1.8370048e-03,  6.4761922e-02]], dtype=float32)

In [33]:
for tree in ensemble:
    pred += tree.predict(np.array([[2,0.3,0.1]]))

In [35]:
pred.reshape(-1)

array([2.3327122, 3.7699025], dtype=float32)

In [12]:
#Test nn ensemble

models = [reg_net_2, reg_net_2b, reg_net_2c]

predictions = np.zeros(len(y_test))

for model in models:
    pred = model(torch.as_tensor(np.array(X_test), dtype=torch.float32))
    pred = pred.detach().numpy().squeeze() * 100
    print(sklearn.metrics.r2_score(y_test, pred))
    predictions += pred

f_pred = predictions/3

sklearn.metrics.r2_score(y_test, f_pred)

0.3822276280658584
0.3460596307384649
0.33965134938161246


0.38001340395836725

In [70]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:,-1:])

In [117]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [113]:
params = {'objective': 'reg:linear', 'verbose': False}
model_1 = xgb.train(params, dtrain, 30)

[22:41:48] WARNING: c:\users\dev-admin\croot2\xgboost-split_1675461376218\work\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:41:48] WARNING: C:\Users\dev-admin\croot2\xgboost-split_1675461376218\work\src\learner.cc:767: 
Parameters: { "verbose" } are not used.



In [119]:
print(sklearn.metrics.r2_score(model_1.predict(dtest), y_test))

-1.8390318864985389


In [ ]:
reg.fit(X_train, y_train)

In [ ]:
reg.score(X_train, y_train)

In [ ]:
reg.score(X_test, y_test)

In [46]:
reg.save_model('boostedLarge.json')

In [28]:
pred = reg_net_2(torch.as_tensor(np.array(X_test), dtype=torch.float32))

In [29]:
pred = pred * 100

In [130]:
pred = reg_net_2(torch.as_tensor(np.array(X_test), dtype=torch.float32))
pred = pred * 100
sklearn.metrics.r2_score(y_test, pred.detach().numpy())

0.5357155496499317

In [25]:
if pred is None:
    print('yes')

In [128]:
pred.detach().numpy().squeeze()

array([  -2.3956685,   34.64344  ,   32.753227 , ...,   26.775593 ,
       -100.       ,   21.044806 ], dtype=float32)

In [140]:
y_test

,value
583669,27.099785
173925,46.437770
605310,0.000000
68553,57.505989
338129,98.338634
...,...
30024,0.000000
434987,0.000000
45223,0.000000
283492,27.589306


In [145]:
pred[:20]

array([25.98377953, 39.89802303,  1.61306703, 71.43904305, 30.69635973,
       27.64241533,  6.11303363, -1.22052059, 24.18680458, 42.36759567,
       25.37599125, 31.93653307, 39.33320293, 34.69208317, 30.05631599,
        6.99665217, -3.01039083, 20.48804369, 38.48104877, -0.10076074])

In [22]:
len(X_train)

599819

In [14]:
reg_net_2b

Sequential(
  (0): Linear(in_features=9, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=32, bias=True)
  (3): ReLU()
  (4): Linear(in_features=32, out_features=32, bias=True)
  (5): ReLU()
  (6): Linear(in_features=32, out_features=32, bias=True)
  (7): ReLU()
  (8): Linear(in_features=32, out_features=1, bias=True)
  (9): Tanh()
)

In [5]:
cdf

,cue_x,cue_y,one_x,one_y,two_x,two_y,speed,vspin,hspin
0,0.581036,0.740190,0.268824,0.843442,0.953960,0.861517,1.85,0.30,0.02
1,0.307287,0.182782,0.292776,1.400334,0.835003,1.645083,1.40,0.00,0.02
2,0.437656,1.911626,0.039922,0.584923,0.151116,1.469425,2.75,0.10,0.18
3,0.357712,1.486969,0.344829,1.197008,0.881224,0.922519,2.15,-0.05,0.02
4,0.646680,1.140687,0.487560,0.806790,0.740952,1.765699,2.45,-0.45,0.02


In [75]:
nums=[1,2]
nums[:-2]

[]